In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

## preparing the data

In [2]:
# data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'
# !wget $data -O data-week-3.csv 

In [3]:
df = pd.read_csv('data-week-3.csv')

In [4]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

df = df.applymap(
    lambda x: x.lower().replace(' ', '_') if (isinstance(x, str)) else x
)

In [5]:
df['totalcharges']= pd.to_numeric(
    df.totalcharges, errors='coerce'
).fillna(0)

In [6]:
df['churn'] = (df.churn == 'yes').astype(int)

## setting the train-val-test dataset

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
df_full_train, df_test = train_test_split(df, test_size=0.20, random_state=1)

In [9]:
## define test size to be the 20% of the full df, you can do it by
# 20%/80% = 1/4
df_train, df_val = train_test_split(
    df_full_train, test_size=0.25, random_state=1
)

In [10]:
len(df_test), len(df_val), len(df_test)

(1409, 1409, 1409)

In [11]:
# avoid the random index
df_train.reset_index(drop=True, inplace=True)
df_val.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

In [12]:
## get the rarget variable
y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

In [13]:
del df_train['churn']
del df_val['churn']
del df_test['churn']

## EDA

In [14]:
df_full_train.reset_index(drop=True, inplace=True)

In [15]:
## verify for null values
df_full_train.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [16]:
df_full_train.churn.value_counts(normalize=True)

0    0.730032
1    0.269968
Name: churn, dtype: float64

In [17]:
global_churn_rate = df_full_train.churn.mean()
round(global_churn_rate, 2)

0.27

In [18]:
# 3 numerical variables and all the other categorical
df_full_train.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges        float64
churn                 int64
dtype: object

In [19]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']
categorical = [
    'gender', 'seniorcitizen', 'partner', 'dependents', 'phoneservice', 
    'multiplelines', 'internetservice', 'onlinesecurity', 'onlinebackup',
    'deviceprotection', 'techsupport', 'streamingtv', 'streamingmovies', 
    'contract', 'paperlessbilling', 'paymentmethod'
]

In [20]:
## how many options we have for each categoical?
df_full_train[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

## Feature importance: Difference and Risk Ratio

In [23]:
## see the churn rate for gender
churn_female = df_full_train[df_full_train.gender == 'female'].churn.mean()
churn_female

0.27682403433476394

In [24]:
churn_male = df_full_train[df_full_train.gender == 'male'].churn.mean()
churn_male

0.2632135306553911

In [28]:
global_churn = df_full_train.churn.mean()
global_churn

0.26996805111821087

In [26]:
## churn rate for partner 
churn_partner = df_full_train[df_full_train.partner == 'yes'].churn.mean()
churn_partner

0.20503330866025166

In [27]:
## churn rate for partner 
churn_no_partner = df_full_train[df_full_train.partner == 'no'].churn.mean()
churn_no_partner
## higher rate for no partner than the golbal

0.3298090040927694

In [29]:
## risk ratio
print(churn_no_partner / global_churn)
print(churn_partner / global_churn)

1.2216593879412643
0.7594724924338315


In [37]:
## we can calculate difference and risk for all the categorical
for c in categorical:
    print(c)
    df_group = df_full_train.groupby(c).churn.agg(['mean', 'count'])
    df_group['diff'] = df_group['mean'] - global_churn
    df_group['risk'] = df_group['mean'] / global_churn
    display(df_group)
    print()
    print()

gender


,mean,count,diff,risk
gender,,,,
female,0.276824,2796,0.006856,1.025396
male,0.263214,2838,-0.006755,0.974980




seniorcitizen


,mean,count,diff,risk
seniorcitizen,,,,
0,0.242270,4722,-0.027698,0.897403
1,0.413377,912,0.143409,1.531208




partner


...

,mean,count,diff,risk
contract,,,,
month-to-month,0.431701,3104,0.161733,1.599082
one_year,0.120573,1186,-0.149395,0.446621
two_year,0.028274,1344,-0.241694,0.104730




paperlessbilling


,mean,count,diff,risk
paperlessbilling,,,,
no,0.172071,2313,-0.097897,0.637375
yes,0.338151,3321,0.068183,1.252560




paymentmethod


,mean,count,diff,risk
paymentmethod,,,,
bank_transfer_(automatic),0.168171,1219,-0.101797,0.622928
credit_card_(automatic),0.164339,1217,-0.105630,0.608733
electronic_check,0.455890,1893,0.185922,1.688682
mailed_check,0.193870,1305,-0.076098,0.718121


## Feature importance: Mutual information

In [38]:
from sklearn.metrics import mutual_info_score

In [39]:
mutual_info_score(df_full_train.contract, df_full_train.churn)

0.0983203874041556

In [40]:
## no much information
mutual_info_score(df_full_train.gender, df_full_train.churn)

0.0001174846211139946

In [41]:
def mutual_info_churn_score(series):
    return mutual_info_score(series, df_full_train.churn)

In [44]:
mi = df_full_train[categorical].apply(mutual_info_churn_score)
mi.sort_values(ascending=False)
# we got the importance of categorical variables

contract            0.098320
onlinesecurity      0.063085
techsupport         0.061032
internetservice     0.055868
onlinebackup        0.046923
deviceprotection    0.043453
paymentmethod       0.043210
streamingtv         0.031853
streamingmovies     0.031581
paperlessbilling    0.017589
dependents          0.012346
partner             0.009968
seniorcitizen       0.009410
multiplelines       0.000857
phoneservice        0.000229
gender              0.000117
dtype: float64

## Feature importance: correlation

In [46]:
## measure importance with numerical variables 
# tenure is permanency in months
df_full_train[numerical].corrwith(df_full_train.churn)

tenure           -0.351885
monthlycharges    0.196805
totalcharges     -0.196353
dtype: float64

In [52]:
df_full_train[df_full_train.tenure <= 2].churn.mean()

0.5953420669577875

In [50]:
df_full_train[
    (df_full_train.tenure > 2) & (df_full_train.tenure <= 12)
].churn.mean()

0.3994413407821229

In [51]:
df_full_train[df_full_train.tenure > 12].churn.mean()

0.17634908339788277

In [53]:
df_full_train[df_full_train.monthlycharges <= 20].churn.mean()

0.08795411089866156

In [54]:
df_full_train[
    (df_full_train.monthlycharges > 20) & (df_full_train.monthlycharges <= 50)
].churn.mean()

0.18340943683409436

In [55]:
df_full_train[df_full_train.monthlycharges > 50].churn.mean()

0.32499341585462205